In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image

import warnings
import os
import shutil
warnings.filterwarnings('ignore')

2024-02-26 12:27:31.781114: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-26 12:27:31.824428: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-26 12:27:31.824463: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-26 12:27:31.825600: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-26 12:27:31.832303: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-26 12:27:31.833094: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [4]:
datagen = ImageDataGenerator(
        rotation_range=75,            # Degree range for random rotations.
        brightness_range=(0.25, 1.75),  # Range for picking a brightness shift value from. (
        shear_range=45,              # "stretches the image" # https://www.quora.com/What-is-shear-in-data-augmentation, angle
        zoom_range=[0.7, 1.3],        # Range for random zoom.
        horizontal_flip=True,
        fill_mode='constant',
        cval=255                     # If there are pixels that are missing, fill them with white colour
)
# channel_shift_range - we don't want pictures to be brighter nor darker
# vertical_flip - we don't want to have our pokemon upside-down
# https://towardsdatascience.com/exploring-image-data-augmentation-with-keras-and-tensorflow-a8162d89b844

In [2]:
DATADIR = "/data"
AUGDATADIR = "/augdata"
CURRENTDIR = os.getcwd()

In [1]:
import cv2

for img in os.listdir(CURRENTDIR + DATADIR):
    if img == ".ipynb_checkpoints":
        continue
    read_img = cv2.imread(os.path.join(CURRENTDIR + DATADIR, img))
    
    black_pixels = (read_img == [0, 0, 0]).all(axis=2)
    read_img[black_pixels] = [255, 255, 255]
    
    converted_img = cv2.cvtColor(read_img, cv2.COLOR_BGR2RGB)
    final_img = converted_img.reshape((1, ) + converted_img.shape)
    i = 0
    
    for batch in datagen.flow(final_img, batch_size=1, save_to_dir=CURRENTDIR+AUGDATADIR, save_prefix=img.split('.')[0], save_format='png'):
        
        i += 1
        if i > 9:
            break

In [16]:
for filename in os.listdir(CURRENTDIR+AUGDATADIR):
    file_path = os.path.join(CURRENTDIR+AUGDATADIR, filename)
    if file_path == '/home/jovyan/workspace/augdata/.ipynb_checkpoints':
        continue
    os.unlink(file_path)

In [18]:
files = os.listdir(CURRENTDIR+AUGDATADIR)

num_files = len([f for f in files if os.path.isfile(os.path.join(CURRENTDIR+AUGDATADIR, f))])

print(num_files)

11196


In [8]:
#!zip -r augmented_pokemon.zip /home/jovyan/workspace/augdata
#!zip -r augmented_pokemon_gen_1.zip /home/jovyan/workspace/generation_1
#!zip -r augmented_pokemon_gen_2.zip /home/jovyan/workspace/generation_2
#!zip -r augmented_pokemon_gen_3.zip /home/jovyan/workspace/generation_3
#!zip -r augmented_pokemon_gen_4.zip /home/jovyan/workspace/generation_4
#!zip -r augmented_pokemon_gen_5.zip /home/jovyan/workspace/generation_5
#!zip -r augmented_pokemon_gen_6.zip /home/jovyan/workspace/generation_6
#!zip -r augmented_pokemon_gen_7.zip /home/jovyan/workspace/generation_7
#!zip -r augmented_pokemon_gen_8.zip /home/jovyan/workspace/generation_8
#!zip -r augmented_pokemon_gen_9.zip /home/jovyan/workspace/generation_9

In [3]:
def get_generation(pokedex_number):
    if 1 <= pokedex_number <= 151:
        return 1
    elif 152 <= pokedex_number <= 251:
        return 2
    elif 252 <= pokedex_number <= 386:
        return 3
    elif 387 <= pokedex_number <= 493:
        return 4
    elif 494 <= pokedex_number <= 649:
        return 5
    elif 650 <= pokedex_number <= 721:
        return 6
    elif 722 <= pokedex_number <= 809:
        return 7
    elif 810 <= pokedex_number <= 905:
        return 8
    else:
        return 9

In [4]:
def make_dirs():
    for i in range(9):
        os.makedirs(os.path.join(CURRENTDIR, f'generation_{str(i+1)}'), exist_ok=True)

In [5]:
def arrange_pokemon_into_generations():
    for filename in os.listdir(CURRENTDIR + AUGDATADIR):
        if '.ipynb' in filename:
            continue
        split_filename = []
        if '-' in filename[:5]:
            split_filename = filename.split('-')
        else:
            split_filename = filename.split('_')
        pokedex_number = split_filename[0]
        pokedex_number = pokedex_number[:-1] if 'f' in pokedex_number else pokedex_number
        generation = get_generation(int(pokedex_number))
        source = os.path.join(CURRENTDIR + AUGDATADIR, filename)
        destination = os.path.join(CURRENTDIR, f"generation_{str(generation)}", filename)
        shutil.move(source, destination)

In [6]:
make_dirs()
arrange_pokemon_into_generations()